# Theoretical Part
---

### 1. Hypothesis Testing – The problem of multiple comparisons [5 points]

The problem of multiple comparisons can be viewed in terms of a Bernoulli experiment over multipe hypothesis tests, in which the Type I Error probability $\alpha$ of each hypothesis test is independent of the the previous tests. We treat the probability of Type I Error as the probability of a success in the Bernoulli experiment. Thus, for any one experiment, we have a probability distribtion governed by the following parameters as follows:

* chance of making a Type I Error: $\alpha$
* chance of not making a Type I Error: $1 - \alpha$

Given this setup, a collection of $m$ hypothesis tests generates a binomial distribution for the chance of observing $k$ Type I Errors. The distribution is governed by the following parameters:

* number of trials = $m$
* number of successes = $k$
* probability of Type I Error = $alpha$

The probability mass functions for the binomial distribution is given by:

$$Pr(k\ |\ n, p) = C_n^k p^k (1-p)^{n-k} = \frac{n!}{k!(n-k)!} p^k (1-p)^{n-k}$$

Given this reasoning, we can answer the questions as follows:

$$P(m^{th}\ experiment\ gives\ significant\ result\ |\ m experiments\ lacking\ power\ to\ reject\ H_0) = Pr(Type\ I\ Error) = \alpha$$

$$P(at\ least\ one\ significant\ result\ |\ m\ experiments\ lacking\ power\ to\ reject\ H_0) = Pr(k=1\ |\ n=m, p=\alpha) = \frac{m!}{(m-1)!} \alpha(1-\alpha)^{m-1}$$



### 2. Bias and unfairness in Interleaving experiments [10 points]
A scenario in which a team-draft interleaving algorithm is insensitive between 2 ranked lists of length 4 is detailed in Hoffman et al. (2011). A similar argument is made here for length 3. The scenario is showed in the figure below.

![caption](files/team-draft-interleaving-insensitivity.png)



### References
HOFMANN, K., WHITESON, S., AND DE RIJKE, M. 2011. A probabilistic method for inferring preferences from
clicks. In Proceedings of the ACM Conference on Information and Knowledge Management (CIKM).

# Experimental Part
---

In [1]:
import collections as cl
import itertools
import math
import random
import numpy as np
import pickle

In [2]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

### Step 1: Simulate Rankings of Relevance for  E  and  P (5 points)

In [3]:
def generate_rank_col(rank_len:int=5, rel_set:tuple=(0,1,2))->list:
    """ Generate collection of all possible rankings.
    
    Args:
        rank_len: Length of each ranking.
        rel_set: Set of relevance scores.
    Returns:
        The cartesian product of of `rel_set` repeated `rank_len` times.
    """
    
    return [x for x in itertools.product(rel_set, repeat=rank_len)]

def generate_rank_pairs(rank_col:list)->list:
    """ Generate all possible pairs of rankings.
    
    Args:
        rank_col: a collection of rankings
    Returns:
        The cartesian product of a ranking repeated twice.
    """
    
    return [Pair(*x) for x in itertools.product(rank_col, repeat=2)]

Pair = cl.namedtuple('Pair', ['P', 'E'])
RANKING_LENGTH = 5
REL_SET = (0,1,2)

rank_pairs = generate_rank_pairs(generate_rank_col(RANKING_LENGTH, REL_SET))

### Step 2 :  Implement Evaluation Measures  (10 points)

In [4]:
### Average Precision
def compute_avg_precision(ranking, denominator_R=10, denominator_HR=10):
    """Compute average of precisions at relevant and highly-relevant documents. 
    
    Args:
        ranking: Relevance ranking
        denominator_R: Total relevant documents in collection
        denominator_HR: Total highly-relevant documents in collection
    Returns:
        Tuple with average of precisions of relevant and highly-relevant documents
    """

    R_precision_sum = 0
    R_docs = 0
    
    HR_precisions_sum = 0
    HR_docs = 0
    
    for i in range(len(ranking)):
        if ranking[i] == 1:
            R_docs += 1
            R_precision_sum += (R_docs / (i + 1))
        elif ranking[i] == 2:
            HR_docs += 1
            HR_precisions_sum += (HR_docs / (i + 1))
    
    return (R_precision_sum / denominator_R, HR_precisions_sum / denominator_HR)

### nDCG@k
def compute_DCG(ranking, k=5):
    """Compute Discounted Cumulative Gain at rank k. 
    
    Args:
        ranking: Relevance ranking
        k: Position on which DCG is computed; by default, 5
    Returns:
        DCG value at rank k
    """
    
    DCG = 0
    for i in range(k):
        DCG += (2 ** ranking[i] - 1) / (math.log2(i + 2))
    
    return DCG

def compute_nDCG(ranking, k=5, maxDCG=1):
    """Compute normalized Discounted Cumulative Gain at rank k. 
    
    Args:
        ranking: Relevance ranking
        k: Position on which DCG is computed; by default, 5
        maxDCG: DCG for best possible ranking for normalisation
    Returns:
        Normalized DCG value at rank k
    """
    
    return compute_DCG(ranking, k) / maxDCG

### ERR
def compute_theta(rel, max_rel=2):
    """Compute probability of satisfaction.
    
    Args:
        rel: Relevance score of certain document
        max_rel: Maximum possible relevance
    Returns:
        Probability of satisfaction
    """
    return (2 ** rel - 1) / (2 ** max_rel)

def compute_ERR(ranking, max_rel=2):
    """Compute Expected Reciprocal Rank.
    
    Args:
        ranking: Relevance ranking
        max_rel: Maximum possible relevance of documents
    Returns:
        Expected Reciprocal Rank measure
    """
    p = 1
    ERR = 0
    
    for i in range(len(ranking)):
        R = compute_theta(ranking[i], max_rel)
        ERR = ERR + p * R / (i + 1)
        p = p * (1 - R)
    
    return ERR

In [5]:
initial_scores = {}
maxDCG = compute_DCG((2, 2, 2, 2, 2))

for pair in rank_pairs:
    pair_scores = {}

    pair_scores['AP_P_R'], pair_scores['AP_P_HR'] = compute_avg_precision(pair.P)
    pair_scores['AP_E_R'], pair_scores['AP_E_HR'] = compute_avg_precision(pair.E)

    pair_scores['nDCG_P'] = compute_nDCG(pair.P, maxDCG=maxDCG)
    pair_scores['nDCG_E'] = compute_nDCG(pair.E, maxDCG=maxDCG)

    pair_scores['ERR_P'] = compute_ERR(pair.P)
    pair_scores['ERR_E'] = compute_ERR(pair.E)

    initial_scores[pair] = pair_scores

### Step 3 :  Calculate the  𝛥measure (0 points)

In [6]:
pair_scores = {}

for key in initial_scores:
    deltas = {}
    score = initial_scores[key]

    if score['AP_E_R'] > score['AP_P_R']:
        deltas['AP_R'] = score['AP_E_R'] - score['AP_P_R']

    if score['AP_E_HR'] > score['AP_P_HR']:
        deltas['AP_HR'] = score['AP_E_HR'] - score['AP_P_HR']

    if score['nDCG_E'] > score['nDCG_P']:
        deltas['nDCG'] = score['nDCG_E'] - score['nDCG_P']

    if score['ERR_E'] > score['ERR_P']:
        deltas['ERR'] = score['ERR_E'] - score['ERR_P']

    if deltas:
        pair_scores[key] = deltas

### Step 4 :  Implement Interleaving ( 15 points )

In [7]:
class Team_Draft_Interleaving():
    def __init__(self):
        # Variables hold the interleaved ranking, and the teams.
        # The items are identified by the index in the interleaved ranking.
        self.rank_i, self.team_e, self.team_p = list(),list(),list()

        # Variable holds the scores of each team.
        self.clicks = {'stream': [], 'scores': {'E': 0, 'P': 0}}
    
    def interleave_pairs(self, pair: Pair) -> (list, list, list):
        """Interleaves a pair of rankings using team draft method.

        Args:
            pair: The pair of experimental and production rankings.
        Returns:
            Interleaved ranking and teams  
        """
        rank_e, rank_p = pair.E, pair.P    
        count_e, count_p = 0, 0

        # Algorithm is implemented according to slides, with some simplifications. Because all documents are
        # assumed unique, the checks become simpler. Every time an item from one rank is added to the interleaved result,
        # the item is added to the team and counter for that list is incremented. When both counters reach the length
        # of the their respective rankings, both lists have been exhausted.
        while count_e < len(rank_e) and count_p < len(rank_p):
            pick_team_e = (len(self.team_e) < len(self.team_p)) or (len(self.team_e) == len(self.team_p) and random.choice((True, False)))
            if pick_team_e:
                self.rank_i.append(rank_e[count_e])
                self.team_e.append(len(self.rank_i)-1)
                count_e += 1
            else:
                self.rank_i.append(rank_p[count_p])
                self.team_p.append(len(self.rank_i)-1)
                count_p += 1
        return self.rank_i[:len(pair.E)], self.team_e, self.team_p
    
    def assign_click(self, index: int) -> str:
        """Assign simulated click to the owner of the item.

        Args:
            index: The index of the item clicked.
        Return:
            A string 'E' or 'P' representing the owner of the item.
        """
        
        self.clicks['stream'].append(index)
        if index in self.team_e:
            self.clicks['scores']['E'] += 1
            return 'E'
        elif index in self.team_p:
            self.clicks['scores']['P'] += 1
            return 'P'
        else:
            raise IndexError('Index {} not in either teams'.format(index))
            
    def reset_clicks_history(self):
        """Reset clicks variable to clear history"""
        self.clicks = {'stream': [], 'scores': {'E': 0, 'P': 0}}

In [8]:
# ranking_pair = Pair((2,1,0,1,1), (1,0,1,2,1)) # actual main would probably loop over the ranking pairs generate at step 1

# team_draft_algo = Team_Draft_Interleaving()
# team_draft_algo.team_draft_interleave(ranking_pair)

# for i in range(5):
#     team_draft_algo.assign_click(random.choice(range(9)))
    
# team_draft_algo.clicks

### Step 5 :  Implement User Clicks Simulation ( 15 points )

#### 1. Random Click model (RCM)

### $\rho = \frac{\sum_{s\in S}\sum_{u\in s}c_{u}^{(s)}}{\sum_{s\in S}\vert S \vert}$

In other words, $\rho = \frac{ \text{number of clicks}}{\text{number of documents shown}}$

In [9]:
train_file = "./resources/YandexRelPredChallenge.txt"

In [10]:
class RCM:
    def __init__(self, log_filename):
        log_file = open(log_filename)
        self.probability = self.get_parameter(log_file)
        
    def _is_querry(self, line):
        return line.split()[2].lower() == 'q'

    def _get_url_list(self, line):
        assert line.split()[2].lower() == 'q'
        return line[5:]
    
    def get_parameter(self, training_data):
        """
        (a) A method that learns the parameters of the model given a set of training data.
        """
        documents_shown = 0
        clicks = 0
        for line in training_data:
            if self._is_querry(line): # is querry
                url_list = self._get_url_list(line)
                number_of_urls = len(url_list)
                documents_shown += number_of_urls
            else:# is click
                clicks += 1
        return clicks / documents_shown
    
    def click_probabilities(self, urls):
        """
        (b) A method that predicts the click probability given a ranked list of relevance labels.
            For RCM, all links have the same probability.
        """
        return [self.probability for i in range(0,len(urls))]
    
    def clicks(self, click_probabilities):
        """
        (c) A method that decides - stochastically - whether a document is clicked based on their probabilities.
        """
        return [np.random.binomial(1, prob) for prob in click_probabilities]

In [11]:
rcm_model = RCM(train_file)

#### 2. Simple Dependent Click Model (SDCM)

In [12]:
class SDCM:
    def __init__(self, log_filename):
        self.MAX_REL = 2
        log_file = open(log_filename)
        self.rank_probabilities = self.get_parameters(log_file)
        self.attractiveness = lambda x: (2**x-1) / 2**self.MAX_REL
        
    def _is_querry(self, line):
        return line[2].lower() == 'q'

    def _get_url_list(self, line):
        assert line[2].lower() == 'q'
        return line[5:]
    
    def get_rank(self, querry, click):
        if click[3] not in querry[5:]: # weird..
            return -1
        else:
            querry = querry[5:]
            return querry.index(click[3])
    
    def get_parameters(self,training_data):
        """
        (a) A method that learns the parameters of the model given a set of training data.
        """
        last_clicked_rank= -1
        last_querry = -1
        
        last_click_rank_counter = cl.Counter()
        click_rank_counter = cl.Counter()
        
        for line in training_data:
            line = line.split()
            if self._is_querry(line): # is querry
                last_querry = line
                if last_clicked_rank != -1:  #the previusly click was the last one
                    last_click_rank_counter[last_clicked_rank] += 1
                    last_clicked_rank = -1  # we counted it, so we 'remove' it.
            else:# is click
                last_clicked_rank = self.get_rank(last_querry, line)
                click_rank_counter[last_clicked_rank] += 1
        # to take into consideration the last click in the log file.
        if last_clicked_rank != -1:
            last_click_rank_counter[last_clicked_rank] += 1
            last_clicked_rank = -1  # we countend, so we 'remove' it.
            
        return 1 - np.array([last_click_rank_counter[r]/click_rank_counter[r] for r in range(0,10)])
    
    def click_probabilities(self, urls):
        """
        (b) A method that predicts the link atractiveness given a list of relevance labels.
        """
        return [self.attractiveness(i) for i in urls]
    
    def clicks(self, atractiveness):
        """
        (c) A method that decides - stochastically - whether a document is clicked based 
            on their atractiveness and probabilities.
        """
        clicks = np.zeros(len(atractiveness))
        for i, a in enumerate(atractiveness):
            if np.random.binomial(1, a) == 1:
                clicks[i] = 1
                if np.random.binomial(1,self.rank_probabilities[i]) == 0: # we should not contiue
                    break
            else:
                clicks[i] = 0
        return clicks.astype(int).tolist()

In [13]:
sdcm_model = SDCM(train_file)

### Step 6 :  Simulate Interleaving Experiment ( 10 points )

In [14]:
def run_simulation(ranking, algorithm, N, model):
    E_wins = P_wins = 0
    
    for k in range(N):
        clicks = model.clicks(model.click_probabilities(ranking))
        
        for i, c in enumerate(clicks):
            if c == 1:
                algorithm.assign_click(i)

        E_score = algorithm.clicks['scores']['E']
        P_score = algorithm.clicks['scores']['P']
        
        if E_score > P_score:
            E_wins += 1
        elif P_score > E_score:
            P_wins += 1
            
        algorithm.reset_clicks_history()
        
    return E_wins / (E_wins + P_wins) if E_wins + P_wins != 0 else 0

In [15]:
N = 100

for test_pair in pair_scores:
    interleaving_algorithm = Team_Draft_Interleaving()
    ranking, e, p = interleaving_algorithm.interleave_pairs(test_pair)
    
    pair_scores[test_pair]['RCM_E_prop'] = run_simulation(ranking=ranking,
                                                          algorithm=interleaving_algorithm,
                                                          N=N,
                                                          model=rcm_model)
    
    pair_scores[test_pair]['SDCM_E_prop'] = run_simulation(ranking=ranking,
                                                           algorithm=interleaving_algorithm,
                                                           N=N,
                                                           model=sdcm_model)
    
print("Measurement done")

Measurement done


#### Save/load scores

In [16]:
save_obj(pair_scores, 'pairs_with_scores')
# pair_scores = load_obj('pairs_with_scores')

### Step 7 :  Results and   Analysis ( 30 points )

In [ ]:
# import step_7